In [1]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error


In [2]:
# Replace 'file_name.csv' with the actual file name
test = pd.read_csv('Data\Test_Data.csv')
train = pd.read_csv('Data\Train_Data.csv')


In [3]:
X_train, oil, gas = train.drop(columns = ['oil', 'gas']), train.oil, train.gas

# Фичи


# Поиск гиперпараметров

In [4]:
import mlflow
import mlflow.sklearn
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
import numpy as np

In [8]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')#r"C:\Users\pechk\Documents\GitHub\GEOMODELHACK24")

In [11]:
# Define the models and their hyperparameter grids
models = {
    'CatBoost': CatBoostRegressor(verbose=False),
    'XGBoost': XGBRegressor(),
    'LightGBM': LGBMRegressor(),
}

hyperparameter_grids = {
    'CatBoost': {
        'iterations': [100, 200, 300],
        'learning_rate': [0.1, 0.05, 0.01],
    },
    'XGBoost': {
        'n_estimators': list(range(10, 301, 50)),
        'learning_rate': [0.1, 0.05, 0.01],
    },
    'LightGBM': {
        'n_estimators': list(range(10, 301, 50)),
        'learning_rate': [0.1, 0.05, 0.01],
    },
}
def start_experiment_grid_search(X_train, y_train, experiment_name):
    # Start the MLflow experiment
    mlflow.set_experiment(experiment_name)

    # Iterate over the models and their hyperparameter grids
    for model_name, model in models.items():
        # Start a new MLflow run for the current model
        with mlflow.start_run(run_name=model_name) as run:
            # Create a GridSearchCV object with custom RMSE scorer
            scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)
            grid_search = GridSearchCV(model, hyperparameter_grids[model_name], cv=5, scoring=scorer)

            # Fit the GridSearchCV object to the training data
            grid_search.fit(X_train, y_train)

            # Get the best hyperparameters and the best score
            best_params = grid_search.best_params_
            best_score = -grid_search.best_score_  # Since we used a negative RMSE scorer

            # Log the best hyperparameters and the best score in MLflow
            mlflow.log_params(best_params)
            mlflow.log_metric("RMSE", best_score)

            # Log the best model and its hyperparameters
            mlflow.sklearn.log_model(grid_search.best_estimator_, model_name, registered_model_name=model_name)

In [12]:
start_experiment_grid_search(X_train, oil, 'Oil v1')

start_experiment_grid_search(X_train, gas, 'Gas v1')

2024/08/06 10:36:40 INFO mlflow.tracking.fluent: Experiment with name 'Oil v1' does not exist. Creating a new experiment.
c:\Users\pechk\Documents\GitHub\GEOMODELHACK24\.conda\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\pechk\Documents\GitHub\GEOMODELHACK24\.conda\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\pechk\Documents\GitHub\GEOMODELHACK24\.conda\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002097 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6641
[LightGBM] [Info] Number of data points in the train set: 12800, number of used features: 27
[LightGBM] [Info] Start training from score 656740.873710
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001861 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6641
[LightGBM] [Info] Number of data points in the train set: 12801, number of used features: 27
[LightGBM] [Info] Start training from score 669142.597880
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the over

c:\Users\pechk\Documents\GitHub\GEOMODELHACK24\.conda\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\pechk\Documents\GitHub\GEOMODELHACK24\.conda\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\pechk\Documents\GitHub\GEOMODELHACK24\.conda\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002643 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6641
[LightGBM] [Info] Number of data points in the train set: 12801, number of used features: 27
[LightGBM] [Info] Start training from score 668197.460042
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003887 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6641
[LightGBM] [Info] Number of data points in the train set: 12801, number of used features: 27
[LightGBM] [Info] Start training from score 665289.359748


c:\Users\pechk\Documents\GitHub\GEOMODELHACK24\.conda\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


...

c:\Users\pechk\Documents\GitHub\GEOMODELHACK24\.conda\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002547 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6641
[LightGBM] [Info] Number of data points in the train set: 12801, number of used features: 27
[LightGBM] [Info] Start training from score 1076257213.609917


c:\Users\pechk\Documents\GitHub\GEOMODELHACK24\.conda\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003294 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6641
[LightGBM] [Info] Number of data points in the train set: 16001, number of used features: 27
[LightGBM] [Info] Start training from score 1094489483.884504


c:\Users\pechk\Documents\GitHub\GEOMODELHACK24\.conda\Lib\site-packages\_distutils_hack\__init__.py:17: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\pechk\Documents\GitHub\GEOMODELHACK24\.conda\Lib\site-packages\_distutils_hack\__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'LightGBM' already exists. Creating a new version of this model...
2024/08/06 11:2

# Сохраняем результаты

In [5]:
# Пример
pd.read_csv('Data\submission_file.csv').head(5)

,ID,oil,gas
0,2308,1,2
1,22404,1,2
2,23397,1,2
3,25058,1,2
4,2664,1,2


In [6]:
results = pd.DataFrame(columns=['ID', 'oil', ' gas'])
results['ID'] = test['ID']
results['oil'] = LGBMRegressor(n_estimators=260, learning_rate=0.1).fit(X_train, oil).predict(test)
results['gas'] = LGBMRegressor(n_estimators=260, learning_rate=0.1).fit(X_train, gas).predict(test)
results.to_csv('Data\Хакатон_Приматы_Задача1.csv', index=False)
results

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003161 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6641
[LightGBM] [Info] Number of data points in the train set: 16001, number of used features: 27
[LightGBM] [Info] Start training from score 663581.963369


ValueError: Number of features of the model must match the input. Model n_features_ is 27 and input n_features is 28

In [17]:
a = [1061120460.9,	928342985.6,	1589664680.5,	1897923970.8]
a

[1061120460.9, 928342985.6, 1589664680.5, 1897923970.8]

In [19]:
min([1061120460.9,	928342985.6,	1589664680.5,	1897923970.8])

928342985.6